# Aprimorando a MLP com Grid Search para escolher o melhor número de camadas intermediárias no Sistema de Lorenz

## 1. Importando as bibliotecas necessárias

### 1.1 Bibliotecas gerais

In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # a biblioteca 'seaborn' contém vários estilos para os gráficos do 'matpĺotlib'

# agora, melhoramos a qualidade de saida e de visualizacao da imagem 
# alem de mudar a fonte padrao para uma do latex
sns.set_style("ticks")
plt.rcParams['savefig.dpi'] = 200
plt.rcParams["figure.dpi"] = 100

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
plt.style.use('dark_background')

### 1.2 Bibliotecas para MLP

In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow import keras
from tensorflow.keras import backend as keras_backend

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [4]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

### 1.3 Bibliotecas dos sistemas caóticos

In [5]:
import sys 
sys.path.insert(0, '../../../../scripts')

import timeseries as times
import lorenzsystem as lsys

## 2. Gerando a série temporal do sistema de Lorenz

Os valores para os parâmetros das equações serão os "básicos" apresentados no *paper* original.

In [6]:
t_inicial = 0
t_final = 50
dt = 0.01

In [7]:
estado_inicial = np.array([0.1, 0, 0])

In [8]:
sis_lorenz = lsys.SistemaLorenz(estado_inicial, dt=dt)

In [9]:
solucoes, instantes_temporais = sis_lorenz.calcular(t_inicial = t_inicial, t_final = t_final)

In [10]:
x = solucoes[:, 0]
y = solucoes[:, 1]
z = solucoes[:, 2]

In [11]:
fig, ax = plt.subplots()
ax.set_title("Série temporal em $\hat{x}$ de 0 a 50 segundos do Sistema de Lorenz\n utilizando $\sigma = 10$, " + r"$\beta =\frac{8}{3}$, " + r"$\rho=28$, com " + "$x(0) =$ " + str(estado_inicial[0]) + ", $y(0) = $ " + str(estado_inicial[1]) + " e $z(0) =$ " + str(estado_inicial[2]))
ax.plot(instantes_temporais, x, color='DodgerBlue')

ax.set_ylabel('$x(t)$')
ax.set_xlabel('$t$')
ax.set_xlim(0,50)
ax.grid(True)

fig.tight_layout()
sns.despine()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.1 Dividindo em um conjunto de treinamento e de teste, para K = 4 e L = 3

In [12]:
K = 4
L = 3
tam_teste = 0.15

In [13]:
serie_temporal = times.SerieTemporal(x, K=K, L=L)

In [15]:
X_treino, X_teste, y_treino, y_teste = serie_temporal.dividir_treino_teste(tam_teste)

## 3. Definindo o modelo para a MLP

### 3.1 Definindo função para criar a MLP

In [27]:
def criar_modelo(input_size, optimizer='Nadam', batch_normalization='OFF', learning_rate=0.001, activation='selu', init_mode='lecun_normal', n_neurons=30,
                 n_hidden_layers=1, loss="mean_squared_error", name="MLP"):
    
    if (optimizer=='Nadam'):
        model_optimizer = keras.optimizers.Nadam()
    
    model_optimizer.learning_rate.assign(learning_rate)
    
    model = keras.Sequential(name=name)
    model.add(keras.layers.Dense(input_size, input_dim=input_size, name="camada_de_entrada", activation = 'linear'))
    if (batch_normalization == 'ON'):
        model.add(keras.layers.BatchNormalization(name="camada_de_batch_normalization"))
    for i in range(0, n_hidden_layers):
        model.add(keras.layers.Dense(n_neurons, input_dim=input_size, activation=activation, kernel_initializer=init_mode, name="camada_intermediaria_"+str(i+1)))
    model.add(keras.layers.Dense(1, activation='linear', name="camada_de_saida"))
    
    model.compile(
        optimizer = model_optimizer,
        loss = loss)
    
    model.build()
    return model

### 3.2 Definindo parâmetros que não serão definidos pelo *Grid Search*

In [17]:
loss = "mean_squared_error"
optimizer = 'Nadam'
batch_size = 2
batch_normalization = 'OFF'
activation = 'selu'
init_mode = 'lecun_normal'
n_neurons = 50
learning_rate = 0.001
input_size = K

### 3.3 Executando *Grid Search* para saber o melhor número de camadas intermediárias

In [18]:
model_cv = KerasRegressor(build_fn=criar_modelo, epochs=100, verbose=0, batch_size=batch_size, batch_normalization=batch_normalization,
                          learning_rate=learning_rate, activation=activation, init_mode=init_mode, n_neurons=n_neurons,
                          loss=loss, optimizer=optimizer, input_size=input_size)

In [19]:
param_grid = {'n_hidden_layers' : (1, 2, 3, 4, 5)}

In [20]:
grid = GridSearchCV(estimator=model_cv, param_grid=param_grid, n_jobs=1, cv=4, scoring='neg_mean_squared_error', verbose=1)

In [21]:
grid_result = grid.fit(X_treino, y_treino)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


In [22]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.004630 using {'n_hidden_layers': 4}


In [23]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

-0.006590 (0.003658) with: {'n_hidden_layers': 1}
-0.010946 (0.006145) with: {'n_hidden_layers': 2}
-0.006404 (0.004295) with: {'n_hidden_layers': 3}
-0.004630 (0.003092) with: {'n_hidden_layers': 4}
-0.010009 (0.007319) with: {'n_hidden_layers': 5}


### Treino com o melhor modelo

In [24]:
n_hidden_layers=4

In [28]:
model = criar_modelo(name="MLP-Lorenz", input_size=K, optimizer=optimizer, batch_normalization=batch_normalization, learning_rate=learning_rate, activation=activation,
                     init_mode=init_mode, n_neurons=n_neurons, n_hidden_layers=n_hidden_layers, loss=loss)

In [29]:
model.summary()

Model: "MLP-Lorenz"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
camada_de_entrada (Dense)    (None, 4)                 20        
_________________________________________________________________
camada_intermediaria_1 (Dens (None, 50)                250       
_________________________________________________________________
camada_intermediaria_2 (Dens (None, 50)                2550      
_________________________________________________________________
camada_intermediaria_3 (Dens (None, 50)                2550      
_________________________________________________________________
camada_intermediaria_4 (Dens (None, 50)                2550      
_________________________________________________________________
camada_de_saida (Dense)      (None, 1)                 51        
Total params: 7,971
Trainable params: 7,971
Non-trainable params: 0
______________________________________________________

In [30]:
X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, test_size=0.1)

In [31]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss')

In [33]:
history = model.fit(X_treino, y_treino, epochs=100,
                            callbacks=early_stopping, validation_data=(X_val, y_val),
                            batch_size=batch_size)
treinamento = pd.DataFrame(history.history)

Epoch 1/100
1910/1910 [==============================] - 5s 2ms/step - loss: 1.6718 - val_loss: 0.3673
Epoch 2/100
1910/1910 [==============================] - 5s 2ms/step - loss: 0.1560 - val_loss: 0.5482
Epoch 3/100
1910/1910 [==============================] - 4s 2ms/step - loss: 0.2118 - val_loss: 0.4967
Epoch 4/100
1910/1910 [==============================] - 4s 2ms/step - loss: 0.1070 - val_loss: 0.1124
Epoch 5/100
1910/1910 [==============================] - 4s 2ms/step - loss: 0.1246 - val_loss: 0.0205
Epoch 6/100
1910/1910 [==============================] - 4s 2ms/step - loss: 0.0869 - val_loss: 0.0128
Epoch 7/100
1910/1910 [==============================] - 4s 2ms/step - loss: 0.0317 - val_loss: 0.0248
Epoch 8/100
1910/1910 [==============================] - 3s 2ms/step - loss: 0.1006 - val_loss: 0.0749
Epoch 9/100
1910/1910 [==============================] - 4s 2ms/step - loss: 0.0724 - val_loss: 0.0403
Epoch 10/100
1910/1910 [==============================] - 4s 2ms/step - l

## Teste com o melhor modelo

In [34]:
y_pred = model.predict(X_teste)

### Gráfico

In [35]:
fig, ax = plt.subplots()
ax.plot(instantes_temporais[len(instantes_temporais)-int(len(instantes_temporais)*tam_teste):,], y_teste, color='DodgerBlue', label='Valor real')
ax.plot(instantes_temporais[len(instantes_temporais)-int(len(instantes_temporais)*tam_teste):,], y_pred, color='Silver', label='MLP')

ax.set_title("Comparação da predição da MLP com o valor real do sistema de Lorenz\n utilizando a rede neural ótima")
ax.set_ylabel('$x(t)$')
ax.set_xlabel('$t$')
ax.set_xlim(42.5, 50)
    
ax.grid(True)
sns.despine()
ax.legend()

plt.show()
fig.savefig("../../../../images/mlp-aprimorada/performance/mlp-aprimorada-vs-lorenz.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

findfont: Font family ['serif'] not found. Falling back to DejaVu Sans.


### Erro Quadrático Médio

In [36]:
from sklearn.metrics import mean_squared_error

In [37]:
mse = mean_squared_error(y_teste, y_pred)
print("MSE = " + str(mse))

MSE = 0.0024601999625926824


### Salvando o Modelo

In [38]:
model.save("../../../../models/mlp-aprimorada/mlp-aprimorada-lorenz.h5", include_optimizer=True)